In [ ]:
#Create oct6 spacing source spaces for fsaverage individual subjects
import os
import mne
aalto_recon_all = r"D:\REFTEP_ALL\REFTEP_reco\Aalto_recon_all/"
os.environ['SUBJECTS_DIR'] = aalto_recon_all
fsaverage_dir = mne.datasets.fetch_fsaverage(subjects_dir=aalto_recon_all)
recon_all_dir = r"D:\REFTEP_ALL\REFTEP_reco"
spacing='oct6'
os.environ['SUBJECTS_DIR'] = "None"

#first create it for fsaverage
fname_fsaverage_src = os.path.join(aalto_recon_all,'fsaverage','fsaverage-oct-6-src.fif')
if not os.path.isfile(fname_fsaverage_src):
    src_to = mne.setup_source_space(subject='fsaverage', spacing=spacing, subjects_dir=aalto_recon_all) #set up the source space for fsaverage
    mne.write_source_spaces(fname_fsaverage_src, src_to, overwrite=True) #save the src file
    
#then for individual subjects for non-fatsuppressed T1 MRIs
for mri_dir in os.listdir(recon_all_dir):
    if 'fatsuppressed' not in mri_dir and 'recon_all' in mri_dir:
        subjects_mri_directory = os.path.join(recon_all_dir,mri_dir)
        site = mri_dir.split("_")[0]
        subjects_eeg_directory = rf"D:\REFTEP_ALL\EEG_preprocessing_data\Preprocessing_{site}"
        for subject in os.listdir(subjects_mri_directory):
            if subject != 'fsaverage':
                fname_src = os.path.join(subjects_mri_directory,subject,f'{subject}-oct-6-src.fif')
                src = mne.setup_source_space(subject=subject, spacing=spacing, subjects_dir=subjects_mri_directory) #set up the source space for the subject (surface source space)
                mne.write_source_spaces(fname_src, src, overwrite=True) #save the src file

In [ ]:
import mne
import os
#create forward models for each subject based on the BEM surfaces, source-spaces, and head--mri trasformations

eeg_std = 0.1 #default value for adhoc cov (in microvolts)
recon_all_dir = r"D:\REFTEP_ALL\REFTEP_reco"
for mri_dir in os.listdir(recon_all_dir):
    if 'fatsuppressed' not in mri_dir:
        subjects_mri_directory = os.path.join(recon_all_dir,mri_dir)
        site = mri_dir.split("_")[0]
        source_site = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
        for subject in os.listdir(subjects_mri_directory):
            if subject == "sub-105_reco" or subject == "sub-110_reco" or subject == "sub-118_reco" or subject == "sub-120_reco":
                reftep_subject = subject.split("_reco")[0]
                subject_reftep_now = [name for name in os.listdir(source_site) if reftep_subject in name]
                if len(subject_reftep_now) not in [0,1]:
                    print("Too many identifiers for this subject")
                    break
                elif len(subject_reftep_now) == 0:
                    print(f'did not find subject {reftep_subject} in {source_site}')
                else:
                    reftep_subject = subject_reftep_now[0]
                    sourcepath_subject = os.path.join(source_site,reftep_subject)
                    subject_mri_dir = os.path.join(subjects_mri_directory,subject)#the subject mri directory in subjects_mri_directory
                    epochs_file = os.path.join(sourcepath_subject,f'{reftep_subject}_final_eeg-epo.fif') #epochs file with digitization coordinates
                    epochs = mne.read_epochs(epochs_file)
                    transpath = os.path.join(sourcepath_subject,f'{reftep_subject}_coreg',f'{reftep_subject}-trans.fif') #eeg-mri coregistration transformation matrix
                    bempath = os.path.join(subject_mri_dir, 'bem','bem-solution')
                    bem_solution = mne.read_bem_solution(bempath) #load the bem solution
                    fname_src = os.path.join(subjects_mri_directory,subject,f'{subject}-oct-6-src.fif')
                    src = mne.read_source_spaces(fname_src) #read source spaces
                    fwd = mne.make_forward_solution(epochs.info, transpath, src, bem_solution, meg=False, eeg=True) #compute the forward solution
                    fwd_fname = os.path.join(sourcepath_subject, f'{reftep_subject}-fwd.fif')
                    mne.write_forward_solution(fwd_fname, fwd, overwrite=True)
                    cov = mne.make_ad_hoc_cov(epochs.info, std={'eeg':eeg_std*1e-6}) #make an ad-hoc covarince matrix
                    cov_fname = os.path.join(sourcepath_subject, f'{reftep_subject}_adhoc-cov.fif')
                    cov.save(cov_fname, overwrite=True)